# Road Usage Analysis Notebook
This notebook walks you through the work flow for how road usage data on the Dangermond Preserve was collected, analyzed, and published. This script will require you to point certain variables to relative paths on your machine. As such, running it without setting these paths will fail.

## Include Required Python Packages and Libraries

In [ ]:
import arcpy
import os
import glob
import pandas as pd
import numpy as np
import geopandas as gpd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.gis import GIS
from shapely import Point
import ipdb
import dotenv

## Use .env to store and reference local paths

In [ ]:
from dotenv import load_dotenv

load_dotenv()
PROJECT_DIR = dotenv.get_key(".env", key_to_get = "PROJECT_DIR")

## Attach notebook to your ArcGIS Project
This notebook references an already created ArcGIS Pro Project. In order to run this script properly, please create an ArcGIS Project with the two following layers added:

- 1st - JLDP Roads: https://tnc.maps.arcgis.com/home/item.html?id=000625f21f054e93b67ef44bd4026a49
- 2nd - JLDP LoRa Vehicle Tracking: https://tnc.maps.arcgis.com/home/item.html?id=f77e630cfe914483929f2a0bfba230c3

- Additionally, store the JLDP Boundary shapefile in your project folder: https://tnc.box.com/s/0f4unpe3dmd6b5uhxpo4z65idv1qnmx6

If you do not have permissions to access these datasets please contact: Jinsu.elhance@tnc.org with information regarding your usage and institution. 

In [ ]:
project_dir = f"{PROJECT_DIR}/FinalRoadUse/"
aprx = arcpy.mp.ArcGISProject(f"{project_dir}/FinalRoadUse.aprx")

try:
    mp = aprx.listMaps()[0]
except IndexError:
    print("Please add a map to your project")
    
arcpy.env.workspace = f"{project_dir}/your_geodatabase.gdb" #replace with your project's geodatabase path

## Create or Identify Directories for GeoJSON and Shapefile Outputs
To move files out of the ArcGIS file types and read them into Geopandas (the library used here for spatial analysis) the files must be converted into geojsons. In order to add the final outputs of our analysis back to our project, we convert a geodataframe to a shapefile and place it in the project's geodatabase. 

In [ ]:
#Check if directories for storing the intermittent geojsons and shapefiles exist, if not, create them. 
#These can be found in your ArcGIS Project directory. 
if not os.path.exists(f"{project_dir}/GeoJSONs"):
    os.makedirs(f"{project_dir}/GeoJSONs")
geojson_dir = f"{project_dir}/GeoJSONs"
    
if not os.path.exists(f"{project_dir}/Shapefiles"):
    os.makedirs(f"{project_dir}/Shapefiles")
shapefile_dir = f"{project_dir}/Shapefiles"

# Useful Functions

In [ ]:
#Helper functions for manipulating arcpy layers and geodataframes.

def getFields(layer):
    """
    Inputs: 
    layer (arcpy._mp.Layer): Input layer with data table to pull fields from.
    Outputs:
    list (arcpy._mp.Field): a list of fields from the input layer data table.
    """
    assert type(layer) == arcpy._mp.Layer
    _dsc = arcpy.da.Describe(layer)
    if _dsc.get('children', False):
        print("This layer is a grouped layer")
        return None
    return arcpy.da.Describe(layer)['fields']

def getFieldNames(layer):
    """
    Inputs: 
    layer (arcpy._mp.Layer): Input layer with data table to pull fields from.
    Outputs:
    list (string): a list of field names from the input layer data table.
    """
    assert type(layer) == arcpy._mp.Layer
    _dsc = arcpy.da.Describe(layer)
    if _dsc.get('children', False):
        print("This layer is a grouped layer")
        return None
    return [field.name for field in arcpy.da.Describe(layer)['fields']]

def LayerToGDF(project, layer, crs):
    """
    Input: 
    project (arcpy._mp.ArcGISProject) : Current working project
    layer (arc._mp.Layer) : layer with data to be converted to a geodataframe
    crs (string) : EPSG crs that you want layer to be projected to. Defaults to 'EPSG:4326'
    Output:
    returns geopandas geodataframe from layer
    """
    assert type(project) == arcpy._mp.ArcGISProject and geojson_dir
    output_file = arcpy.da.Describe(layer)['aliasName']
    
    if (os.path.exists(f"{geojson_dir}/{output_file}.geojson")):
        _owrite = input(f"This GeoJSON ({output_file}) has already been created, would you like to overwite (y/n)").lower()
        if _owrite == "y":
            os.remove(f"{geojson_dir}/{output_file}.geojson")
            LayerToGDF(project, layer, crs)
        else:
            return gpd.GeoDataFrame.from_file(f"{geojson_dir}/{output_file}.geojson").to_crs(crs)

    arcpy.conversion.FeaturesToJSON(layer, f"{geojson_dir}/{output_file}.geojson", geoJSON = True)
    return gpd.GeoDataFrame.from_file(f"{geojson_dir}/{output_file}.geojson").to_crs(crs)

def GDFToLayer(gdf, out_features, geometry_type, project):
    """
    Inputs:
    gdf (GeoDataFrame)
    out_features (string)
    geometry_type (string)
    project (arcpy._mp.ArcGISProject)
    Output:
    None -> Prints status message
    """
    #Assert Geojson dir and shapefile dir are defined
    assert geojson_dir and shapefile_dir
    
    if (os.path.exists(f"{shapefile_dir}/{out_features}.shp")):
        _owrite = input(f"This Shapefile ({out_features}) has already been created, would you like to overwite (y/n)").lower()
        if _owrite == "y":
            os.remove(f"{geojson_dir}/{out_features}.geojson")
            for f in glob.glob(f"{shapefile_dir}/{out_features}.*"):
                os.remove(f)
            GDFToLayer(gdf, out_features, geometry_type, project)
        else: 
            arcpy.conversion.FeatureClassToGeodatabase(f"{shapefile_dir}/{out_features}.shp", project.defaultGeodatabase)
            return
        
    else: #Create new GeoJSON and shp
        #Write GeoDataFrame to a GeoJSON in the geojson_dir 
        gdf.to_file(f"{geojson_dir}/{out_features}.geojson", driver="GeoJSON")
        #Read new GeoJSON to a shapefile
        arcpy.conversion.JSONToFeatures(f"{geojson_dir}/{out_features}.geojson", f"{shapefile_dir}/{out_features}", geometry_type = geometry_type)
    
    #Add shapefile to project as a layer
    arcpy.conversion.FeatureClassToGeodatabase(f"{shapefile_dir}/{out_features}.shp", project.defaultGeodatabase)
    
    print("Successfully added to geodatabase")
    return

# Perform Analysis

### Pull Layers into GeoDataFrames
Use the helper functions defined above to read in layers from the project as geodataframes. 
Ensure the indices used in line 2 and 3 are correctly pointing to the roads and lora tracking layers.

In [ ]:
layer_list = [layer.name for layer in mp.listLayers()]
roads_jldp = mp.listLayers()[0]
lora_tracking_jldp = mp.listLayers()[1]

roads_jldp_gdf = LayerToGDF(aprx, roads_jldp, "EPSG:4326")
lora_jldp_gdf = LayerToGDF(aprx, lora_tracking_jldp, "EPSG:4326")

### Clip Data to JLDP Boundary

In [ ]:
jldp_bounds = gpd.read_file("JLDP_BOUNDARY_SHAPEFILE_DIR.shp") #Replace with the location of the JLDP Boundary you downloaded.

#Clip data to the JLDP Boundary
lora_jldp_gdf_clipped = gpd.clip(lora_jldp_gdf, jldp_bounds)
roads_jldp_gdf_clipped = gpd.clip(roads_jldp_gdf, jldp_bounds)

### Generate Near Table
Associate LoRa tracking points to the nearest road, with a search limit of 20 meters.

In [ ]:
try:
    arcpy.analysis.GenerateNearTable(lora_tracking_jldp, roads_jldp, "lora_road_near_table", "20 meters", closest=True)
except:
    print("File already exists")

In [ ]:
near_table_ptr = arcpy.ListTables("lora_road_near_table")[0]
_fieldnames = [field.name for field in arcpy.da.Describe(near_table_ptr)['fields']]

### Joining Lora Tracking Data to Roads
Create a geodataframe from the near table to perform a spatial join between the lora tracking point data and the roads data.

In [ ]:
near_table = gpd.GeoDataFrame(arcpy.da.TableToNumPyArray("lora_road_near_table", _fieldnames))

In [ ]:
_lora_road_join = (lora_jldp_gdf_clipped
    .merge(near_table, left_on = "FID", right_on="IN_FID", how="left")
    .rename(columns={'NEAR_FID':'road_ID'}))

_lora_road_join['date'] = _lora_road_join['rec_tm_utc'].dt.floor("D")

daily_road_use = (_lora_road_join
                  .groupby(['road_ID', 'date', 'dev'])[['road_ID', 'date', 'dev']]
                  .agg(lambda x: None or x))

#Collate all points which belong to the same unique combination of (road ID, vehicle, and date)
daily_road_use['month'] = _lora_road_join['date'].dt.strftime('%Y/%m')
daily_road_use['year'] = _lora_road_join['date'].dt.strftime('%Y')

#Generate periodic summaries of road usage
monthly_road_use = (daily_road_use
                    .groupby(['road_ID', 'month'])[['dev']]
                    .count()
                    .rename(columns = {"dev":'vehicle_count'})
                    .reset_index())

yearly_road_use = (daily_road_use
                   .groupby(['road_ID', 'year'])[['dev']]
                   .count()
                   .rename(columns = {"dev":'vehicle_count'})
                   .reset_index())

total_road_use = (daily_road_use
                  .groupby(['road_ID'])[['dev']]
                  .count()
                  .rename(columns = {"dev": 'vehicle_count'})
                  .reset_index())

In [ ]:
#Preview total road usage
total_road_use_lines = gpd.GeoDataFrame(roads_jldp_gdf_clipped
                       .merge(total_road_use, left_on="FID", right_on="road_ID", how="left")
                       .set_index('road_ID'))

total_road_use_lines.head()

In [ ]:
#Preview monthly summary of road usage
monthly = (roads_jldp_gdf_clipped
    .merge(monthly_road_use, left_on="FID", right_on="road_ID", how="right")[['road_ID', 'geometry', 'month', 'vehicle_count']]
    .sort_values('month')
    .set_index('road_ID'))

monthly.head()

### Write Data Back to ArcGIS Project

In [ ]:
#Use helpers to add monthly and total road use data to the ArcGIS project.
GDFToLayer(monthly, "monthly_road_use", "POLYLINE", aprx)
GDFToLayer(total_road_use_lines, "total_road_use", "POLYLINE", aprx)